In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer 
from nltk.stem import PorterStemmer
#nltk.download('stopwords')
from nltk.corpus import stopwords

#!python -m spacy download en_core_web_sm
import spacy
sp = spacy.load('en_core_web_sm')

import re

2023-02-16 12:05:25.973079: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 12:05:26.382526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-16 12:05:26.382549: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-16 12:05:27.393534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [19]:
stop_words = set(stopwords.words('english'))

#getting a library of stopwords and defining a lemmatizer
porter=SnowballStemmer("english")
lmtzr = WordNetLemmatizer()

In [20]:
df = pd.read_csv("tds.csv")

In [21]:
df

,Unnamed: 0,id,url,title,subtitle,n_sections,n_paragraphs,section_titles,story_text,claps,responses,reading_time,publication,date,year
0,0,1,https://towardsdatascience.com/difference-betw...,Difference between Permutation and Combination,NaN,1,25,['Difference between Permutation and Combinati...,Long story short The difference between Permut...,479.0,5,3,Towards Data Science,2017-05-31,2017
1,1,2,https://towardsdatascience.com/building-a-real...,Building a realtime dashboard with Flink: The ...,NaN,1,9,['Building a realtime dashboard with Flink: Th...,With the demand for “realtime” low latency dat...,16.0,0,3,Towards Data Science,2017-05-31,2017
2,2,3,https://towardsdatascience.com/artificial-inte...,Artificial Intelligence is the Panacea to Toda...,NaN,12,27,['Artificial Intelligence is the Panacea to To...,"My foray into understanding, and more importan...",33.0,3,7,Towards Data Science,2017-05-31,2017
3,3,4,https://towardsdatascience.com/opportunities-a...,"<span class=""markup--anchor markup--h3-anchor""...",NaN,1,6,['Opportunities And Obstacles For Deep Learnin...,Target audience: general. 27 scientists collab...,NaN,0,3,Towards Data Science,2017-05-31,2017
4,4,5,https://towardsdatascience.com/https-medium-co...,The Single Most Important Thing That Data Can ...,How to reduce the cognitive load that entrepre...,1,22,['The Single Most Important Thing That Data Ca...,I’ve worked with a number of startups and ther...,83.0,0,4,Towards Data Science,2017-05-31,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,813,814,https://towardsdatascience.com/a-low-down-on-m...,A Low Down on Machine Learning,NaN,12,68,"['A Low Down on Machine Learning', 'Part 1', '...",How machine learning and artificial intelligen...,79.0,1,13,Towards Data Science,2022-02-28,2022
814,814,815,https://towardsdatascience.com/background-task...,Background tasks for NLP,Heavy lifting is better in the background rath...,3,33,"['Background tasks for NLP', 'Recap', 'Heavy l...",Those people dancing in the photo are similar ...,0.0,0,8,Towards Data Science,2022-02-28,2022
815,815,816,https://towardsdatascience.com/elasticsearch-w...,Elasticsearch Workshop #6 — Scripting Part 4,Regex and pattern matching,9,27,['Elasticsearch Workshop #6 — Scripting Part 4...,"Welcome to part 6 of the workshop. As usual, t...",15.0,0,5,Towards Data Science,2022-02-28,2022
816,816,817,https://towardsdatascience.com/data-science-tr...,Data Science training — run them effectively,A few leads and watch-outs as you prepare to t...,2,50,['Data Science training — run them effectively...,Data Science and Analytics is a very rapidly e...,55.0,0,8,Towards Data Science,2022-02-28,2022


In [22]:
#the following leaves in place two or more capital letters in a row
#will be ignored when using standard stemming
def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()

In [25]:
#modular pipeline for stemming, lemmatizing and lowercasing
#note this is NOT lemmatizing using grammar pos
def tokenize(text, modulation):
    tokens = re.split(r'\W+', text)
    stems = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        lowers=abbr_or_lower(token)
        if lowers not in stop_words:
            if re.search('[a-zA-Z]', lowers):
                if modulation==0:
                    stems.append(lowers)
                if modulation==1:
                    stems.append(porter.stem(lowers))
                if modulation==2:
                    stems.append(lmtzr.lemmatize(lowers))
                stems.append(" ")
    return "".join(stems)

In [29]:
def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector

In [34]:
corpus = []
for text in df.title:
    processed_text = tokenize(text, 1)
    corpus.append(processed_text)

In [36]:
%%time
#build vocabulary
vocab, index = {}, 1  # start indexing from 1
for doc in corpus:
    #I added this line from lecture to add tokenization
    tokens = doc.split()
    for token in tokens:
      if token not in vocab:
        vocab[token] = index
        index += 1
        
vocab_stem=vocab
vocab_stem_size = len(vocab)
print(vocab_stem)
print(" ")
print("Total size of stemmed vocabulary:", vocab_stem_size)

vectors=[]
for doc in corpus:
    vectors.append(vectorize(doc, vocab_stem))

df = pd.DataFrame(vectors)
df

{'differ': 1, 'permut': 2, 'combin': 3, 'build': 4, 'realtim': 5, 'dashboard': 6, 'flink': 7, 'backend': 8, 'artifici': 9, 'intellig': 10, 'panacea': 11, 'today': 12, 'market': 13, 'challeng': 14, 'span': 15, 'class': 16, 'markup': 17, 'anchor': 18, 'h3': 19, 'data': 20, 'action': 21, 'open': 22, 'inner': 23, 'link': 24, 'valu': 25, 'https': 26, 'www': 27, 'linkedin': 28, 'com': 29, 'puls': 30, 'opportun': 31, 'obstacl': 32, 'deep': 33, 'learn': 34, 'biolog': 35, 'medicin': 36, 'johnni': 37, 'isra': 38, 'singl': 39, 'import': 40, 'thing': 41, 'startup': 42, 'probabl': 43, 'weight': 44, 'coin': 45, 'flip': 46, 'game': 47, 'use': 48, 'python': 49, 'numpi': 50, 'right': 51, 'answer': 52, 'bias': 53, 'statist': 54, 'problem': 55, 'scienc': 56, 'divers': 57, 'linear': 58, 'algebra': 59, 'cheat': 60, 'sheet': 61, 'kalman': 62, 'filter': 63, 'intuit': 64, 'discret': 65, 'case': 66, 'deriv': 67, 'strong': 68, 'explor': 69, 'ed': 70, 'sheeran': 71, 'song': 72, 'found': 73, 'machin': 74, 'stori'

CPU times: user 445 ms, sys: 16.9 ms, total: 462 ms
Wall time: 455 ms


,0,1,2,3,4,5,6,7,8,9,...,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
815,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
816,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
